In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import requests

# NGROK_DOMAIN = "http://eez115.ece.ust.hk:5000/"
NGROK_DOMAIN = "http://localhost:8080/"
session_id = 405307062


response = requests.post(f"{NGROK_DOMAIN}/get_paf_result", json={"session_id": session_id})
response = response.json()
response_data = response.get("responses", {})
conversation_history = response_data.pop("conversation_history", None)
table = pd.DataFrame.from_dict(response_data, orient='index')

In [3]:
conversation_history

[{'role': 'assistant', 'utterance': '你好啊！我喺 Grace。你今日覺得點啊？'},
 {'role': 'user', 'utterance': "I'm good"},
 {'role': 'assistant', 'utterance': '請問你有冇咳?'},
 {'role': 'user', 'utterance': 'Yes I do'},
 {'role': 'assistant', 'utterance': '請問有冇痰?'},
 {'role': 'user', 'utterance': "No I don't"},
 {'role': 'assistant', 'utterance': '你最近嗰三個月有冇試過跌親?'},
 {'role': 'user', 'utterance': 'yes'},
 {'role': 'assistant', 'utterance': '你使唔使揸拐杖或者扶住嘢行路架?'},
 {'role': 'user', 'utterance': 'yes'},
 {'role': 'assistant', 'utterance': '請問你嘅視力好唔好?睇野清唔清楚呀?'},
 {'role': 'user', 'utterance': 'yes'},
 {'role': 'assistant', 'utterance': '你喺過去三個月有冇無緣無故變瘦?'},
 {'role': 'user', 'utterance': "no i don't"},
 {'role': 'assistant', 'utterance': '胃口點呀?有冇唔想食嘢?'},
 {'role': 'user', 'utterance': 'no'},
 {'role': 'assistant', 'utterance': '好嘅請問你有冇假牙呀?'}]

In [4]:
table

,patient_answer,question_asked,session_id,slot_name,slot_value
q1_0_coughing_existance,Yes I do,請問你有冇咳?,405307062,cough_existence,yes
q2_0_sputum_existance,No I don't,請問有冇痰?,405307062,sputum_existence,no
q3_0_history_falling,yes,你最近嗰三個月有冇試過跌親?,405307062,history_of_falling,yes
q3_1_ambulatory_aid,yes,你使唔使揸拐杖或者扶住嘢行路架?,405307062,ambulatory_aid,crutches/cane/walkers
q4_0_vision,yes,請問你嘅視力好唔好?睇野清唔清楚呀?,405307062,vision_problem,yes
q5_0_lose_weight_existance,no i don't,你喺過去三個月有冇無緣無故變瘦?,405307062,weight_loss_existance,no
q5_2_decreased_appetite,no,胃口點呀?有冇唔想食嘢?,405307062,appetite_loss,no


In [5]:
table.reset_index(drop=True)[["session_id", "slot_name", "question_asked","slot_value"]]

,session_id,slot_name,question_asked,slot_value
0,405307062,cough_existence,請問你有冇咳?,yes
1,405307062,sputum_existence,請問有冇痰?,no
2,405307062,history_of_falling,你最近嗰三個月有冇試過跌親?,yes
3,405307062,ambulatory_aid,你使唔使揸拐杖或者扶住嘢行路架?,crutches/cane/walkers
4,405307062,vision_problem,請問你嘅視力好唔好?睇野清唔清楚呀?,yes
5,405307062,weight_loss_existance,你喺過去三個月有冇無緣無故變瘦?,no
6,405307062,appetite_loss,胃口點呀?有冇唔想食嘢?,no


In [6]:
table.slot_name.unique()

array(['cough_existence', 'sputum_existence', 'history_of_falling',
       'ambulatory_aid', 'vision_problem', 'weight_loss_existance',
       'appetite_loss'], dtype=object)

In [7]:
from fill_PAF_table import fill_paf_table

fill_paf_table(response.get("responses", {}))

,PAF_Question,PAF_Result
0,Coughing 請問有冇咳？,☐ No \t ☑️ Yes
1,Sputum 請問有冇痰？,☑️ No \t ☐ Yes
2,Amount 有痰的話，有幾多量？,
3,Color 痰顏色係咩？,
4,History of Falling 有冇跌過？,☐ No \t ☑️ Yes
5,Ambulatory Aid 有冇用過行動輔助器具？,crutches/cane/walkers
6,Vision Problem 視力問題？,☐ No \t ☑️ Yes
7,Weight Loss 有冇體重下降？,☑️ No \t ☐ Yes
8,Weight Loss Amount 輕咗幾多？,
9,Appetite Loss 胃口差别？,☑️ No \t ☐ Yes


In [8]:
session_info = requests.post(f"{NGROK_DOMAIN}/get_session_info", json={"session_id": session_id})
session_info = session_info.json()

In [9]:
session_info

{'responses': {'curr_question_id': 'q6_0_denture',
  'grace_uttrs_history': ['initial_status',
   '請問你有冇咳?',
   '請問有冇痰?',
   '你最近嗰三個月有冇試過跌親?',
   '你使唔使揸拐杖或者扶住嘢行路架?',
   '請問你嘅視力好唔好?睇野清唔清楚呀?',
   '你喺過去三個月有冇無緣無故變瘦?',
   '胃口點呀?有冇唔想食嘢?',
   '請問你有冇假牙呀?'],
  'history': ['initial_status',
   'q0_greeting',
   'q1_0_coughing_existance',
   'q2_0_sputum_existance',
   'q3_0_history_falling',
   'q3_1_ambulatory_aid',
   'q4_0_vision',
   'q5_0_lose_weight_existance',
   'q5_2_decreased_appetite'],
  'q_tracker': ['initial_status',
   'q0_greeting',
   'q1_0_coughing_existance',
   'q2_0_sputum_existance',
   'q3_0_history_falling',
   'q3_1_ambulatory_aid',
   'q4_0_vision',
   'q5_0_lose_weight_existance',
   'q5_2_decreased_appetite',
   'q6_0_denture'],
  'session_id': 405307062}}